In [770]:
# !pip install selenium

In [632]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import ElementClickInterceptedException
import numpy as np
import pymysql

# 1. Andhra - apsrtc

In [442]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile");

time.sleep(5)

In [443]:
def route_link_name(path):
    route_names = []
    route_links = []
    
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        apsrtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in apsrtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [447]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada, Name: Hyderabad to Vijayawada
Link: https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad, Name: Vijayawada to Hyderabad
Link: https://www.redbus.in/bus-tickets/hyderabad-to-ongole, Name: Hyderabad to Ongole
Link: https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam, Name: Kakinada to Visakhapatnam
Link: https://www.redbus.in/bus-tickets/bangalore-to-tirupathi, Name: Bangalore to Tirupati
Link: https://www.redbus.in/bus-tickets/bangalore-to-kadapa, Name: Bangalore to Kadapa
Link: https://www.redbus.in/bus-tickets/ongole-to-hyderabad, Name: Ongole to Hyderabad
Link: https://www.redbus.in/bus-tickets/kadapa-to-bangalore, Name: Kadapa to Bangalore
Link: https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore, Name: Chittoor (Andhra Pradesh) to Bangalore
Link: https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada, Name: Visakhapatnam to Kakinada
Link: https://www.redbus.in/bus-tickets/bangalore

In [448]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("andhra_route_details.csv")
df.shape

(46, 2)

In [449]:
def scrolling_a():
    scrolling = True
    while scrolling:
        old_page_source = driver_a.page_source
        driver_a.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_page_source = driver_a.page_source
        if new_page_source == old_page_source:
            scrolling = False

In [450]:
driver_a = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_a, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_a.get(route_link)
    time.sleep(2)
    
    route_links = driver_a.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)
        
    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_a()

    bus_name = driver_a.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_a.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_a.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_a.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_a.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_a.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_a.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_a.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [451]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

5290 5290 5290 5290 5290 5290 5290 5290 5290 5290


In [452]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_andhra = pd.DataFrame(dict)

In [453]:
df_andhra

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:50,05h 45m,05:35,4.6,INR 410,42 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),04:35,05h 55m,10:30,4.7,450,30 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),05:50,05h 55m,11:45,4.7,450,33 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdShyamoli Paribahan Pvt Ltd,Scania Multi-Axle AC Semi Sleeper (2+2),23:25,05h 45m,05:10,4.4,INR 383.9,45 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),22:35,06h 40m,05:15,4.7,332,29 Seats available
...,...,...,...,...,...,...,...,...,...,...
5285,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri Vaari Travels,A/C Sleeper (2+1),19:00,06h 00m,01:00,3.3,INR 1700,21 Seats available
5286,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri Vaari Travels,A/C Sleeper (2+1),20:40,04h 35m,01:15,3.3,INR 1400,24 Seats available
5287,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,DHANUNJAYA TRAVELS,A/C Sleeper (2+1),19:10,07h 50m,03:00,2.2,INR 1500,36 Seats available
5288,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,DHANUNJAYA TRAVELS,A/C Sleeper (2+1),21:00,06h 00m,03:00,2.2,INR 1000,36 Seats available


In [457]:
df_andhra[df_andhra['Route_name']=='Hyderabad to Vijayawada']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:50,05h 45m,05:35,4.6,INR 410,42 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),04:35,05h 55m,10:30,4.7,450,30 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),05:50,05h 55m,11:45,4.7,450,33 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdShyamoli Paribahan Pvt Ltd,Scania Multi-Axle AC Semi Sleeper (2+2),23:25,05h 45m,05:10,4.4,INR 383.9,45 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),22:35,06h 40m,05:15,4.7,332,29 Seats available
...,...,...,...,...,...,...,...,...,...,...
368,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Elegance Tours And Travels Pvt Ltd,Bharat Benz A/C Sleeper (2+1),20:35,06h 10m,02:45,1.9,1520,23 Seats available
369,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Sai Durga Travels,A/C Sleeper (2+1),23:05,05h 55m,05:00,1.6,INR 5000,1 Seat available
370,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,SSK Travels,Non A/C Seater / Sleeper (2+1),22:50,05h 25m,04:15,1.6,INR 449,46 Seats available
371,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Sai Durga Travels,A/C Seater / Sleeper (2+1),22:35,05h 55m,04:30,1.6,INR 5000,2 Seats available


In [458]:
df_andhra.to_csv('Andhra.csv')

# 2. Kerala - Kerala RTC

In [512]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile");

time.sleep(5)

In [562]:
def route_link_name(path):
    route_names = []
    route_links = []
    
    for page in range(1,6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        krtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in krtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [566]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/bangalore-to-kozhikode, Name: Bangalore to Kozhikode
Link: https://www.redbus.in/bus-tickets/kozhikode-to-bangalore, Name: Kozhikode to Bangalore
Link: https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam, Name: Kozhikode to Ernakulam
Link: https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode, Name: Ernakulam to Kozhikode
Link: https://www.redbus.in/bus-tickets/bangalore-to-kannur, Name: Bangalore to Kannur
Link: https://www.redbus.in/bus-tickets/kozhikode-to-mysore, Name: Kozhikode to Mysore
Link: https://www.redbus.in/bus-tickets/kannur-to-bangalore, Name: Kannur to Bangalore
Link: https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram, Name: Kozhikode to Thiruvananthapuram
Link: https://www.redbus.in/bus-tickets/mysore-to-kozhikode, Name: Mysore to Kozhikode
Link: https://www.redbus.in/bus-tickets/bangalore-to-kalpetta, Name: Bangalore to Kalpetta (kerala)
Link: https://www.redbus.in/bus-tickets/kalpetta-to-bangalore, Name: 

In [567]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("kerala_route_details.csv")
df.shape

(15, 2)

In [568]:
def scrolling_k():
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        driver_k.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [569]:
driver_k = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_k, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_k.get(route_link)
    time.sleep(2)
    
    route_links = driver_k.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_k()

    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_k.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_k.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_k.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Successfully scraped using Selenium


In [570]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

526 526 526 526 526 526 526 526 526 526


In [571]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_kerala = pd.DataFrame(dict)
df_kerala           #514

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1586,Super Express Non AC Seater Air Bus (2+2),21:31,09h 05m,06:36,3.6,INR 640,1 Seat available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 2133,Swift Deluxe Non AC Air Bus (2+2),22:12,09h 24m,07:36,4.2,INR 640,2 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1064,Super Fast Non AC Seater (2+3),23:45,10h 05m,09:50,3.4,INR 473,31 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,10h 00m,09:00,4.2,INR 799,26 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,08h 25m,06:10,4.2,INR 749,10 Seats available
...,...,...,...,...,...,...,...,...,...,...
521,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,EMERALD TRAVELS,Bharat Benz A/C Semi Sleeper (2+2),23:00,05h 30m,04:30,3.2,INR 700,26 Seats available
522,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,A1 Travels,A/C Seater / Sleeper (2+1),23:59,07h 01m,07:00,2.2,INR 700,30 Seats available
523,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Sonia Travels,A/C Seater/Sleeper (2+1),19:10,07h 19m,02:29,4.0,INR 750,20 Seats available
524,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Ubc travels,A/C Seater/Sleeper (2+1),18:00,09h 00m,03:00,3.7,INR 900,15 Seats available


In [572]:
df_kerala[df_kerala['Route_name']=='Bangalore to Kozhikode']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1586,Super Express Non AC Seater Air Bus (2+2),21:31,09h 05m,06:36,3.6,INR 640,1 Seat available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 2133,Swift Deluxe Non AC Air Bus (2+2),22:12,09h 24m,07:36,4.2,INR 640,2 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1064,Super Fast Non AC Seater (2+3),23:45,10h 05m,09:50,3.4,INR 473,31 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,10h 00m,09:00,4.2,INR 799,26 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,08h 25m,06:10,4.2,INR 749,10 Seats available
5,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,MMK Travels,A/C Seater / Sleeper (2+1),22:30,09h 00m,07:30,4.4,INR 750,16 Seats available
6,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,P K Travels,A/C Sleeper (2+1),18:45,11h 45m,06:30,4.3,INR 1100,5 Seats available
7,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,NS Transports,VE A/C Sleeper (2+1),22:45,08h 45m,07:30,4.1,INR 1099,12 Seats available
8,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,08h 10m,06:10,4.4,INR 599,30 Seats available
9,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,SAAM BUS,VE A/C Sleeper (2+1),20:10,10h 35m,06:45,4.4,INR 800,5 Seats available


In [573]:
df_kerala.to_csv('Kerala.csv')

# 3. Goa - KTCL

In [420]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile");

time.sleep(5)

In [421]:
def route_link_name(path):
    route_names = []
    route_links = []
    
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        ktcl_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in ktcl_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [423]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/pune-to-goa, Name: Pune to Goa
Link: https://www.redbus.in/bus-tickets/goa-to-pune, Name: Goa to Pune
Link: https://www.redbus.in/bus-tickets/mumbai-to-goa, Name: Mumbai to Goa
Link: https://www.redbus.in/bus-tickets/bangalore-to-goa, Name: Bangalore to Goa
Link: https://www.redbus.in/bus-tickets/goa-to-bangalore, Name: Goa to Bangalore
Link: https://www.redbus.in/bus-tickets/goa-to-mumbai, Name: Goa to Mumbai
Link: https://www.redbus.in/bus-tickets/pandharpur-to-goa, Name: Pandharpur to Goa
Link: https://www.redbus.in/bus-tickets/goa-to-pandharpur, Name: Goa to Pandharpur
Link: https://www.redbus.in/bus-tickets/solapur-to-goa, Name: Solapur to Goa
Link: https://www.redbus.in/bus-tickets/calangute-goa-to-goa-airport, Name: Calangute (goa) to Goa Airport
Link: https://www.redbus.in/bus-tickets/goa-to-kolhapur-maharashtra, Name: Goa to Kolhapur(Maharashtra)
Link: https://www.redbus.in/bus-tickets/goa-to-solapur, Name: Goa to Solapur
Link: https://w

In [424]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("goa_route_details.csv")
df.shape

(34, 2)

In [425]:
def scrolling_kt():
    scrolling = True
    while scrolling:
        old_page_source = driver_kt.page_source
        driver_kt.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_kt.page_source
        
        if new_page_source == old_page_source:
            scrolling = False
        else:
            old_page_source = new_page_source

In [426]:
driver_kt = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_kt, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_kt.get(route_link)
    time.sleep(2)
    
    route_links = driver_kt.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_kt()

    bus_name = driver_kt.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_kt.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_kt.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_kt.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_kt.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_kt.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_kt.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_kt.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with 

In [427]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

531 531 531 531 531 531 531 531 531 531


In [428]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_goa = pd.DataFrame(dict)
df_goa                                             #531

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,19:00,10h 00m,05:00,4.0,INR 600,28 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,10h 45m,07:45,4.6,INR 750,12 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Amrutyog,A/C Sleeper (2+1),22:00,10h 00m,08:00,4.4,INR 899,27 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.4,INR 850,8 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:00,12h 00m,09:00,4.3,INR 899,22 Seats available
...,...,...,...,...,...,...,...,...,...,...
526,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,00h 31m,16:01,3.7,INR 100,41 Seats available
527,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,11h 30m,20:00,3.5,INR 400,17 Seats available
528,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),03:30,04h 30m,08:00,1.3,INR 1050,15 Seats available
529,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa,Shirdi Xpress,A/C Sleeper (2+1),08:15,03h 15m,11:30,2.2,INR 419,15 Seats available


In [429]:
df_goa[df_goa['Route_name']=='Pune to Goa']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,19:00,10h 00m,05:00,4.0,INR 600,28 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,10h 45m,07:45,4.6,INR 750,12 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Amrutyog,A/C Sleeper (2+1),22:00,10h 00m,08:00,4.4,INR 899,27 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.4,INR 850,8 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:00,12h 00m,09:00,4.3,INR 899,22 Seats available
...,...,...,...,...,...,...,...,...,...,...
58,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Shubhra Sharma Tourist Services,A/C Sleeper (2+1),00:10,10h 21m,10:31,3.1,INR 1399,10 Seats available
59,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Online go,NON A/C Sleeper (2+1),21:35,11h 00m,08:35,2.8,INR 777,29 Seats available
60,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Online go,NON A/C Sleeper (2+1),20:05,11h 45m,07:50,2.8,INR 777,29 Seats available
61,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,VRL Travels,A/C Sleeper (2+1),23:30,10h 15m,09:45,3.2,INR 800,19 Seats available


In [430]:
df_goa.to_csv('Goa.csv')

# 4. West Bengal - WBTC (CTC)

In [401]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile");

time.sleep(5)

In [402]:
def route_link_name(path):
    route_names = []
    route_links = []
    
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        wbtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in wbtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [409]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/digha-to-barasat-west-bengal, Name: Digha to Barasat (West Bengal)
Link: https://www.redbus.in/bus-tickets/durgapur-to-kolkata, Name: Durgapur to Calcutta
Link: https://www.redbus.in/bus-tickets/digha-to-kolkata, Name: Digha to Calcutta
Link: https://www.redbus.in/bus-tickets/kolkata-to-digha, Name: Kolkata to Digha
Link: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha, Name: Barasat (West Bengal) to Digha
Link: https://www.redbus.in/bus-tickets/kolkata-to-suri, Name: Kolkata to Suri
Link: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-midnapore, Name: Barasat (West Bengal) to Midnapore
Link: https://www.redbus.in/bus-tickets/midnapore-to-kolkata, Name: Midnapore to Kolkata
Link: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-kolaghat, Name: Barasat (West Bengal) to Kolaghat
Link: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-contai-kanthi, Name: Barasat (West Bengal) to Contai (Kanthi)
Link: https://

In [410]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("westbengal_route_details.csv")
df.shape

(38, 2)

In [411]:
def scrolling_w():
    scrolling = True
    while scrolling:
        old_page_source = driver_w.page_source
        driver_w.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_w.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [413]:
driver_w = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_w, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_w.get(route_link)
    time.sleep(2)
    
    route_links = driver_w.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_w()

    bus_name = driver_w.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_w.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_w.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_w.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_w.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_w.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_w.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_w.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [414]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

885 885 885 885 885 885 885 885 885 885


In [415]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_westbengal = pd.DataFrame(dict)
df_westbengal

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Esplanade - 176|04:00,Non AC Seater (2+3),04:00,05h 50m,09:50,1.7,INR 165,32 Seats available
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|05:40,Non AC Seater (2+3),05:40,04h 30m,10:10,3.4,INR 161,20 Seats available
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BADURIA - DIGHA via Bally - 155|09:00,Non AC Seater (2+3),09:00,03h 25m,12:25,3.3,INR 161,5 Seats available
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Bally - 26|10:00,Non AC Seater (2+3),10:00,04h 45m,14:45,3.4,INR 161,50 Seats available
4,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,"WBTC (CTC) BARASAT-DIGHA via Esplanade, Karuna...",Non AC Seater (2+3),10:15,05h 20m,15:35,3.0,INR 165,47 Seats available
...,...,...,...,...,...,...,...,...,...,...
880,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC) HABRA-DIGHA via Bally - 26|06:55,Non AC Seater (2+3),06:55,03h 50m,10:45,4.0,INR 130,47 Seats available
881,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,03h 55m,16:40,4.0,INR 130,51 Seats available
882,Midnapore to Kolkata Airport,https://www.redbus.in/bus-tickets/midnapore-to...,WBTC (CTC) Habra-Midnapur - 194|14:45,Non AC Seater (2+3),14:45,02h 55m,17:40,4.0,INR 111,50 Seats available
883,Midnapore to Kolkata Airport,https://www.redbus.in/bus-tickets/midnapore-to...,WBTC (CTC) Habra-Midnapur - 194|15:40,Non AC Seater (2+3),15:40,03h 00m,18:40,3.8,INR 111,45 Seats available


In [418]:
df_westbengal[df_westbengal['Route_name']=='Barasat (West Bengal) to Kolaghat']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
586,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|03:45,Non AC Seater (2+3),03:45,01h 40m,05:25,5.0,INR 69,38 Seats available
587,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|04:30,Non AC Seater (2+3),04:30,01h 35m,06:05,4.3,INR 69,41 Seats available
588,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:00,Non AC Seater (2+3),05:00,01h 35m,06:35,4.0,INR 69,29 Seats available
589,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:35,Non AC Seater (2+3),05:35,01h 35m,07:10,2.5,INR 69,36 Seats available
590,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|06:00,Non AC Seater (2+3),06:00,01h 35m,07:35,3.3,INR 69,41 Seats available
591,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) Habra-Midnapur - 194|06:25,Non AC Seater (2+3),06:25,01h 40m,08:05,3.6,INR 69,37 Seats available
592,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) Habra-Midnapur - 194|07:00,Non AC Seater (2+3),07:00,01h 45m,08:45,3.3,INR 69,21 Seats available
593,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) Habra- Barasat- Haldia - 180|07:10,Non AC Seater (2+3),07:10,01h 20m,08:30,4.8,INR 69,40 Seats available
594,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) MADHYAMGRAM - DIGHA via Belghoria -...,Non AC Seater (2+3),07:15,01h 30m,08:45,3.6,INR 69,45 Seats available
595,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|08:15,Non AC Seater (2+3),08:15,01h 30m,09:45,1.8,INR 69,47 Seats available


In [419]:
df_westbengal.to_csv('West_bengal.csv')

# 5. SouthBengal - SBSTC

In [574]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile");

time.sleep(5)

In [575]:
def route_link_name(path):
    route_names = []
    route_links = []
    
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        ctu_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in ctu_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [583]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/durgapur-to-kolkata, Name: Durgapur to Calcutta
Link: https://www.redbus.in/bus-tickets/kolkata-to-burdwan, Name: Kolkata to Burdwan
Link: https://www.redbus.in/bus-tickets/haldia-to-kolkata, Name: Haldia to Calcutta
Link: https://www.redbus.in/bus-tickets/kolkata-to-haldia, Name: Kolkata to Haldia
Link: https://www.redbus.in/bus-tickets/kolkata-to-durgapur, Name: Kolkata to Durgapur (West Bengal)
Link: https://www.redbus.in/bus-tickets/kolkata-to-arambagh-west-bengal, Name: Kolkata to Arambagh (West Bengal)
Link: https://www.redbus.in/bus-tickets/midnapore-to-kolkata, Name: Midnapore to Kolkata
Link: https://www.redbus.in/bus-tickets/kolkata-to-digha, Name: Kolkata to Digha
Link: https://www.redbus.in/bus-tickets/digha-to-kolkata, Name: Digha to Calcutta
Link: https://www.redbus.in/bus-tickets/kolkata-to-bankura, Name: Kolkata to Bankura
Link: https://www.redbus.in/bus-tickets/kolkata-to-midnapore, Name: Kolkata to Midnapore
Link: https://www.re

In [584]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("southbengal_route_details.csv")
df.shape

(41, 2)

In [585]:
def scrolling_s():
    scrolling = True
    while scrolling:
        old_page_source = driver_s.page_source
        driver_s.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_s.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [586]:
driver_s = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_s, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_s.get(route_link)
    time.sleep(2)
    
    route_links = driver_s.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_s()

    bus_name = driver_s.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_s.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_s.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_s.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_s.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_s.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_s.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_s.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with 

In [587]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

254 254 254 254 254 254 254 254 254 254


In [588]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_southbengal = pd.DataFrame(dict)
df_southbengal

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-BANKURA - KARUNAMOYEE - A/C - 15:15 - 1500,AC Seater (2+2),17:00,04h 00m,21:00,2.0,INR 300,33 Seats available
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KARUNAMOYEE - 16:10 (BEL DEPOT...,Non AC Seater (2+3),17:25,04h 00m,21:25,3.5,INR 155,45 Seats available
2,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KOLKATA - 16:30 (HOWRAH DEPOT)...,Non AC Seater (2+3),17:40,03h 30m,21:10,4.1,INR 150,50 Seats available
3,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KARUNAMOYEE - 16:40 (ASANSOL D...,Non AC Seater (2+3),17:50,04h 10m,22:00,3.8,INR 155,48 Seats available
4,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo B11R Multi Axle Seater (2+2),16:00,04h 00m,20:00,4.3,INR 486,33 Seats available
...,...,...,...,...,...,...,...,...,...,...
249,Durgapur (West Bengal) to Purulia,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-KOLKATA - PURULIA - VIA - ASN - 14:15 (P...,Non AC Seater (2+3),17:15,06h 45m,00:00,3.7,INR 112,46 Seats available
250,Durgapur (West Bengal) to Bankura,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-KARUNAMOYEE - BANKURA - A/C - 15:15 (BNK...,AC Seater (2+2),19:00,01h 00m,20:00,2.9,INR 80,35 Seats available
251,Durgapur (West Bengal) to Bankura,https://www.redbus.in/bus-tickets/durgapur-to-...,Dolphin tours and travels,A/C Seater / Sleeper (2+2),17:45,01h 15m,19:00,3.8,INR 896,19 Seats available
252,Kolkata to Ashari,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - MIDNAPUR - 18:00 (MID DEPOT) -...,Non AC Seater (2+3),18:00,06h 00m,00:00,4.2,INR 82,47 Seats available


In [601]:
df_southbengal[df_southbengal['Route_name']=='Kolkata to Nandakumar (west bengal)']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
207,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-KOLKATA - NANDIGRAM - 16:45 (HWH DEPOT) ...,Non AC Seater (2+3),16:45,07h 15m,00:00,3.3,INR 85,49 Seats available
208,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-KOLKATA - HALDIA - 17:00 (Haldia Depot) ...,Non AC Seater (2+3),17:00,07h 00m,00:00,4.1,INR 85,48 Seats available
209,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-KOLKATA - HALDIA - 17.30 (HALDIA DEPOT) ...,Non AC Seater (2+3),17:30,06h 30m,00:00,4.1,INR 85,46 Seats available
210,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-KOLKATA - HALDIA - 18.15 (HALDIA DEPOT) ...,Non AC Seater (2+3),18:15,05h 45m,00:00,4.2,INR 85,45 Seats available
211,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,SBSTC-GARIA - DIGHA - 22:30 (DIGHA DEPOT) - 3745,Non AC Seater (2+3),23:45,00h 15m,00:00,4.3,INR 85,50 Seats available
212,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,Basu Travels,A/C Seater Push Back (2+3),23:30,02h 15m,01:45,2.7,INR 300,20 Seats available
213,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,Basu Travels,A/C Seater / Sleeper (2+2),23:59,02h 01m,02:00,2.7,INR 300,33 Seats available
214,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,Subhadip Travels(Kojagar),Non AC Seater (2+3),16:50,02h 25m,19:15,New,399,31 Seats available
215,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,Sagufta Travels (Santosh),A/C Seater / Sleeper (2+2),01:20,02h 00m,03:20,3.3,324,20 Seats available
216,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,Basu Travels,A/C Seater Push Back (2+3),23:55,02h 20m,02:15,2.7,INR 469,21 Seats available


In [590]:
df_southbengal.to_csv('South_Bengal.csv')

# 6. Rajasthan - RSRTC

In [591]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile");

time.sleep(5)

In [592]:
def route_link_name(path):
    route_names = []
    route_links = []
    
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        pepsu_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in pepsu_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [594]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/udaipur-to-jodhpur, Name: Udaipur to Jodhpur
Link: https://www.redbus.in/bus-tickets/jodhpur-to-ajmer, Name: Jodhpur to Ajmer
Link: https://www.redbus.in/bus-tickets/beawer-to-jaipur, Name: Beawar (Rajasthan) to Jaipur (Rajasthan)
Link: https://www.redbus.in/bus-tickets/sikar-to-jaipur, Name: Sikar to Jaipur (Rajasthan)
Link: https://www.redbus.in/bus-tickets/jaipur-to-jodhpur, Name: Jaipur (Rajasthan) to Jodhpur
Link: https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-jaipur, Name: Aligarh (uttar pradesh) to Jaipur (Rajasthan)
Link: https://www.redbus.in/bus-tickets/jaipur-to-aligarh-uttar-pradesh, Name: Jaipur (Rajasthan) to Aligarh (uttar pradesh)
Link: https://www.redbus.in/bus-tickets/jodhpur-to-beawer, Name: Jodhpur to Beawar (Rajasthan)
Link: https://www.redbus.in/bus-tickets/jaipur-to-pilani, Name: Jaipur (Rajasthan) to Pilani
Link: https://www.redbus.in/bus-tickets/kishangarh-to-jaipur, Name: Kishangarh to Jaipur (Rajasthan)
Link

In [595]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
df.to_csv("rajasthan_route_details.csv")
df.shape

(18, 2)

In [596]:
def scrolling_r():
    scrolling = True
    while scrolling:
        old_page_source = driver_r.page_source
        driver_r.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_page_source = driver_r.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [597]:
driver_r = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_r, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_r.get(route_link)
    time.sleep(2)
    
    route_links = driver_r.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_r()

    bus_name = driver_r.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_r.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_r.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_r.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_r.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_r.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_r.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_r.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [598]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

737 737 737 737 737 737 737 737 737 737


In [599]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_rajasthan = pd.DataFrame(dict)
df_rajasthan

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 193271,Express Non AC Seater 2+3,19:30,06h 30m,02:00,4.2,INR 294,41 Seats available
1,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 189622,Express Non AC Seater 2+3,22:00,07h 00m,05:00,3.5,INR 294,41 Seats available
2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,Shree Devnarayan Travels,Bharat Benz A/C Seater (2+1),17:00,04h 30m,21:30,4.5,INR 550,10 Seats available
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,M R Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),17:10,04h 45m,21:55,4.4,INR 600,39 Seats available
4,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,Jain travels regd,A/C Seater / Sleeper (2+1),21:30,07h 00m,04:30,4.1,INR 460,29 Seats available
...,...,...,...,...,...,...,...,...,...,...
732,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Ganesh Travels,A/C Seater / Sleeper (2+1),21:00,04h 30m,01:30,2.8,INR 699,25 Seats available
733,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Geetanjali Travels,NON AC Seater / Sleeper 2+1,23:00,05h 30m,04:30,2.5,INR 280,31 Seats available
734,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,New shiv mahima travels express & cargo,NON A/C Sleeper (2+1),19:15,04h 50m,00:05,1.8,INR 650,30 Seats available
735,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,A One Travel Agency,A/C Seater / Sleeper (2+1),19:30,04h 44m,00:14,1.6,INR 600,42 Seats available


In [600]:
# df_punjab[df_punjab['Route_name']=='Delhi to Jalandhar']

In [602]:
df_rajasthan.to_csv('Rajasthan.csv')

# 7. TELANGANA  - TSRTC

In [617]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile");

time.sleep(5)

In [618]:
def route_link_name(path):
    route_names = []
    route_links = []
    
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        nbtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in nbtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [628]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada, Name: Hyderabad to Vijayawada
Link: https://www.redbus.in/bus-tickets/khammam-to-hyderabad, Name: Khammam to Hyderabad
Link: https://www.redbus.in/bus-tickets/hyderabad-to-khammam, Name: Hyderabad to Khammam
Link: https://www.redbus.in/bus-tickets/hyderabad-to-srisailam, Name: Hyderabad to Srisailam
Link: https://www.redbus.in/bus-tickets/karimnagar-to-hyderabad, Name: Karimnagar to Hyderabad
Link: https://www.redbus.in/bus-tickets/hyderabad-to-karimnagar, Name: Hyderabad to Karimnagar
Link: https://www.redbus.in/bus-tickets/hyderabad-to-mancherial, Name: Hyderabad to Mancherial
Link: https://www.redbus.in/bus-tickets/hyderabad-to-nirmal, Name: Hyderabad to Nirmal
Link: https://www.redbus.in/bus-tickets/hyderabad-to-adilabad, Name: Hyderabad to Adilabad
Link: https://www.redbus.in/bus-tickets/hyderabad-to-ongole, Name: Hyderabad to Ongole
Link: https://www.redbus.in/bus-tickets/kothagudem-to-hyderabad, Name: Kothagudem to

In [629]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("telangana_route_details.csv")
df.shape

(26, 2)

In [630]:
def scrolling_t():
    scrolling = True
    while scrolling:
        old_page_source = driver_t.page_source
        driver_t.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_t.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [633]:
driver_t = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_t, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_t.get(route_link)
    time.sleep(2)
    
    route_links = driver_t.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(3)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException,ElementClickInterceptedException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_t()

    bus_name = driver_t.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_t.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_t.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_t.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_t.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_t.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_t.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_t.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [634]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

1172 1172 1172 1172 1172 1172 1172 1172 1172 1172


In [635]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_telangana = pd.DataFrame(dict)
df_telangana

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 95703,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",20:45,06h 45m,03:30,3.3,INR 420,24 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3544,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",20:45,06h 55m,03:40,4.1,INR 469,26 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3697,STAR LINER(NON-AC SLEEPER 2+1),20:45,07h 00m,03:45,3.9,INR 607,25 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35212,STAR LINER(NON-AC SLEEPER 2+1),20:45,07h 00m,03:45,3.9,INR 607,22 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3883,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",20:50,07h 15m,04:05,3.4,INR 469,32 Seats available
...,...,...,...,...,...,...,...,...,...,...
1167,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri Krishna Travels,Non A/C Seater / Sleeper (2+1),20:15,08h 15m,04:30,2.0,INR 500,5 Seats available
1168,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Vihari Travels,NON AC Seater / Sleeper 2+1,20:15,07h 30m,03:45,2.6,474,9 Seats available
1169,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,VIHARI TRAVELS,NON A/C Sleeper (2+1),22:00,07h 00m,05:00,New,1899,2 Seats available
1170,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,KKaveri Travels,Non A/C Seater / Sleeper (2+1),20:45,07h 45m,04:30,2.4,801,3 Seats available


In [ ]:
# df_northbengal[df_northbengal['Route_name']=='Siliguri to Kolkata']

In [636]:
df_telangana.to_csv('Telangana.csv')

# 8. Uttar Pradesh - UPSRTC

In [637]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile");

time.sleep(5)

In [638]:
def route_link_name(path):
    route_names = []
    route_links = []
    
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        bsrtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in bsrtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [641]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/delhi-to-bareilly, Name: Delhi to Bareilly
Link: https://www.redbus.in/bus-tickets/bareilly-to-delhi, Name: Bareilly to Delhi
Link: https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-delhi, Name: Aligarh (uttar pradesh) to Delhi
Link: https://www.redbus.in/bus-tickets/delhi-to-aligarh-uttar-pradesh, Name: Delhi to Aligarh (uttar pradesh)
Link: https://www.redbus.in/bus-tickets/lucknow-to-allahabad, Name: Lucknow to Allahabad
Link: https://www.redbus.in/bus-tickets/lucknow-to-delhi, Name: Lucknow to Delhi
Link: https://www.redbus.in/bus-tickets/delhi-to-farrukhabad-up, Name: Delhi to Farrukhabad (Uttar Pradesh)
Link: https://www.redbus.in/bus-tickets/farrukhabad-up-to-delhi, Name: Farrukhabad (Uttar Pradesh) to Delhi
Link: https://www.redbus.in/bus-tickets/badaun-to-delhi, Name: Badaun to Delhi
Link: https://www.redbus.in/bus-tickets/allahabad-to-lucknow, Name: Allahabad to Lucknow
Link: https://www.redbus.in/bus-tickets/lucknow-to-agra, N

In [642]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("uttarpradesh_route_details.csv")
df.shape

(49, 2)

In [643]:
def scrolling_u():
    scrolling = True
    while scrolling:
        old_page_source = driver_u.page_source
        driver_u.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_u.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [644]:
driver_u = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_u, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_u.get(route_link)
    time.sleep(2)
    
    route_links = driver_u.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException,NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_u()

    bus_name = driver_u.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_u.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_u.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_u.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_u.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_u.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_u.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_u.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [645]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_uttarpradesh = pd.DataFrame(dict)
df_uttarpradesh

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - RKD0153,Ordinary Non AC Seater 2+3,21:00,05h 30m,02:30,3.3,INR 448,52 Seats available
1,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - SJP0210,Ordinary Non AC Seater 2+3,21:12,05h 34m,02:46,3.3,INR 454,52 Seats available
2,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - RKD0125,Shatabdi AC Seater 2+2,21:22,05h 00m,02:22,3.5,INR 598,37 Seats available
3,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - SBD0111,Pink Express AC Seater 2+2,21:31,05h 29m,03:00,3.3,INR 598,41 Seats available
4,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - LKM0402,Shatabdi AC Seater 2+2,21:46,05h 44m,03:30,3.3,INR 604,37 Seats available
...,...,...,...,...,...,...,...,...,...,...
2533,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - SBD0106,Janrath AC Seater 2+2,23:30,05h 30m,05:00,1.7,INR 501,36 Seats available
2534,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,NON AC Seater / Sleeper 2+1,20:30,06h 00m,02:30,2.3,INR 620,45 Seats available
2535,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,NON A/C Seater/ Sleeper (2+1),21:00,06h 30m,03:30,2.3,INR 570,51 Seats available
2536,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,NON A/C Seater/ Sleeper (2+1),22:45,06h 30m,05:15,2.3,INR 600,51 Seats available


In [646]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

2538 2538 2538 2538 2538 2538 2538 2538 2538 2538


In [649]:
df_uttarpradesh.to_csv('UttarPradesh.csv')

# 9. Himachal Pradesh - HRTC

In [462]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile");

time.sleep(5)

In [463]:
def route_link_name(path):
    route_names = []
    route_links = []
    
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        kaac_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in kaac_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [466]:
kaac = "div.route_details a.route"
Route_Names,Route_links = route_link_name(kaac)

Link: https://www.redbus.in/bus-tickets/delhi-to-shimla, Name: Delhi to Shimla
Link: https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh, Name: Chandigarh to Hamirpur (Himachal Pradesh)
Link: https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-chandigarh, Name: Hamirpur (Himachal Pradesh) to Chandigarh
Link: https://www.redbus.in/bus-tickets/shimla-to-delhi, Name: Shimla to Delhi
Link: https://www.redbus.in/bus-tickets/delhi-to-chandigarh, Name: Delhi to Chandigarh
Link: https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-delhi, Name: Hamirpur (Himachal Pradesh) to Delhi
Link: https://www.redbus.in/bus-tickets/chamba-himachal-pradesh-to-chandigarh, Name: Chamba (Himachal Pradesh) to Chandigarh
Link: https://www.redbus.in/bus-tickets/delhi-to-hamirpur-himachal-pradesh, Name: Delhi to Hamirpur (Himachal Pradesh)
Link: https://www.redbus.in/bus-tickets/chandigarh-to-dharamshala, Name: Chandigarh to Dharamshala (Himachal Pradesh)
Link: https://ww

In [477]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
df.to_csv("himachalpradesh_route_details.csv")
df.shape

(35, 2)

In [468]:
def scrolling_hp():
    scrolling = True
    while scrolling:
        old_page_source = driver_hp.page_source
        driver_hp.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_page_source = driver_hp.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [469]:
driver_hp = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_hp, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_hp.get(route_link)
    time.sleep(2)
    
    route_links = driver_hp.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException):
        print("Button not found, continuing with scraping...")
    
    time.sleep(4)
    
    scrolling_hp()
    
    bus_name = driver_hp.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_hp.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_hp.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_hp.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_hp.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_hp.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_hp.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_hp.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [470]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

958 958 958 958 958 958 958 958 958 958


In [474]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_himachalpradesh = pd.DataFrame(dict)
df_himachalpradesh

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 657,Ordinary 3+2 Non AC Seater,11:00,11h 00m,22:00,2.7,INR 512,33 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 515,Ordinary 3+2 Non AC Seater,12:30,10h 20m,22:50,4.2,INR 507,37 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 551,A/C Executive (2+3),15:30,10h 00m,01:30,3.5,INR 632,42 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 525,Ordinary 3+2 Non AC Seater,16:50,10h 10m,03:00,4.2,INR 507,27 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 532,Ordinary 3+2 Non AC Seater,17:59,10h 16m,04:15,2.5,INR 507,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
953,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Holiday Appeal Private Limited,Non AC Seater (2+2),20:00,12h 00m,08:00,3.6,1709,13 Seats available
954,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,HIMALAYAN HOLIDAYS,Volvo A/C Semi Sleeper (2+2),17:30,13h 00m,06:30,3.5,INR 999,11 Seats available
955,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Heera Himachal Holidays,Volvo A/C Semi Sleeper (2+2),17:30,14h 30m,08:00,3.0,INR 1399,29 Seats available
956,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Yatrabybus.com,Bharat Benz A/C Semi Sleeper (2+2),18:00,10h 00m,04:00,2.2,INR 1199,49 Seats available


In [476]:
df_himachalpradesh.to_csv('HimachalPradesh.csv')

# 10. Jammu - JKRTC

In [771]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/jksrtc");

time.sleep(5)

In [729]:
def route_link_name(path):
    route_names = []
    route_links = []
    
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        jkrtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in jkrtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [730]:
page = "div.route_details a.route"
Route_Names,Route_links = route_link_name(page)

Link: https://www.redbus.in/bus-tickets/srinagar-to-jammu, Name: Srinagar to Jammu (j and k)
Link: https://www.redbus.in/bus-tickets/delhi-to-srinagar, Name: Delhi to Srinagar
Link: https://www.redbus.in/bus-tickets/jammu-to-srinagar, Name: Jammu (j and k) to Srinagar
Link: https://www.redbus.in/bus-tickets/katra-to-jammu, Name: Katra (jammu and kashmir) to Jammu (j and k)
Link: https://www.redbus.in/bus-tickets/jammu-to-poonch, Name: Jammu (j and k) to Poonch
Link: https://www.redbus.in/bus-tickets/mendhar-j-k-to-jammu, Name: Mendhar (J & K) to Jammu (j and k)
Link: https://www.redbus.in/bus-tickets/jammu-to-chandigarh, Name: Jammu (j and k) to Chandigarh
Link: https://www.redbus.in/bus-tickets/chandigarh-to-jammu, Name: Chandigarh to Jammu (j and k)
Link: https://www.redbus.in/bus-tickets/kishtwar-to-jammu, Name: Kishtwar to Jammu (j and k)
No more pages to scroll


In [731]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("jammu_route_details.csv")
df.shape

(9, 2)

In [765]:
def scrolling_j():
    scrolling = True
    while scrolling:
        old_page_source = driver_j.page_source
#         body = driver_j.find_element(By.TAG_NAME,'body')
#         body.send_keys(Keys.PAGE_DOWN)
#         ActionChains(driver_j).send_keys(Keys.PAGE_DOWN).perform()
        driver_j.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_page_source = driver_j.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [766]:
driver_j = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_j, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_j.get(route_link)
    time.sleep(2)
    
    route_links = driver_j.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
    
    time.sleep(4)

    scrolling_j()

    bus_name = driver_j.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_j.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_j.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_j.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_j.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_j.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_j.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_j.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [767]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

64 64 64 64 64 64 64 64 64 64


In [768]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_jammu = pd.DataFrame(dict)
df_jammu                  #105

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,City Land Travels,A/C Seater/Sleeper (2+1),19:25,01h 05m,20:30,3.6,INR 400,40 Seats available
1,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,Zingbus Plus,A/C Sleeper (2+1),19:50,01h 30m,21:20,4.3,INR 120,29 Seats available
2,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,Royal Golden,A/C Seater / Sleeper (2+2),23:00,01h 00m,00:00,2.0,440,32 Seats available
3,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),20:00,01h 00m,21:00,3.7,INR 1499,38 Seats available
4,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,SPEED BUS,A/C Volvo B11R Multi-Axle Sleeper (2+1),20:00,01h 30m,21:30,3.7,INR 3500,36 Seats available
...,...,...,...,...,...,...,...,...,...,...
59,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,RK Holiday,A/C Seater / Sleeper (2+2),23:20,08h 40m,08:00,3.1,INR 649,29 Seats available
60,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),20:20,07h 10m,03:30,2.4,INR 799,3 Seats available
61,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,CAPITAL HOLIDAYS,A/C Seater / Sleeper (2+2),23:00,06h 00m,05:00,2.1,719,30 Seats available
62,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),22:25,07h 10m,05:35,2.4,INR 599,38 Seats available


In [769]:
df_jammu[df_jammu['Route_name']=='Jammu (j and k) to Chandigarh']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
5,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Laxmi holidays,VE A/C Seater / Sleeper (2+1),20:50,07h 30m,04:20,4.4,864,4 Seats available
6,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Laxmi holidays,Bharat Benz A/C Sleeper (2+1),21:30,07h 20m,04:50,4.4,640,8 Seats available
7,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Laxmi holidays,Bharat Benz A/C Sleeper (2+1),22:15,06h 55m,05:10,4.4,684,4 Seats available
8,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,AdBaru travels,A/C Seater / Sleeper (2+2),21:45,07h 20m,05:05,4.0,509,7 Seats available
9,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Harikesh Tour N Travels,A/C Sleeper (2+1),21:10,07h 20m,04:30,4.2,433,2 Seats available
10,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Laxmi holidays,Volvo 9600 Multi-Axle A/C Sleeper (2+1),20:15,07h 25m,03:40,4.3,729,17 Seats available
11,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Radhe Travels,Bharat Benz A/C Sleeper (2+1),21:25,07h 15m,04:40,4.0,497,8 Seats available
12,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,AdRadhe Travels,Bharat Benz A/C Sleeper (2+1),21:25,07h 50m,05:15,4.0,497,8 Seats available
13,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,New Pal Travels,A/C Sleeper (2+1),22:00,07h 30m,05:30,4.4,539,6 Seats available
14,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Moon Light Travels Katra,A/C Sleeper (2+1),20:15,08h 05m,04:20,3.9,494,14 Seats available


In [334]:
df_jammu.to_csv('Jammu.csv') 

## Combining all the State DataFrame to one --> insert into Database --> Clean the DATA

In [672]:
Red_bus_df = pd.concat([df_andhra,df_kerala,df_goa,df_westbengal,df_southbengal,df_rajasthan,df_telangana,
                      df_uttarpradesh,df_himachalpradesh,df_jammu],ignore_index=True)

In [673]:
Red_bus_df

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:50,05h 45m,05:35,4.6,INR 410,42 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),04:35,05h 55m,10:30,4.7,450,30 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),05:50,05h 55m,11:45,4.7,450,33 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdShyamoli Paribahan Pvt Ltd,Scania Multi-Axle AC Semi Sleeper (2+2),23:25,05h 45m,05:10,4.4,INR 383.9,45 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),22:35,06h 40m,05:15,4.7,332,29 Seats available
...,...,...,...,...,...,...,...,...,...,...
12991,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,CAPITAL HOLIDAYS,A/C Seater / Sleeper (2+2),21:30,06h 00m,03:30,2.1,539,39 Seats available
12992,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),20:20,07h 10m,03:30,2.4,INR 595,38 Seats available
12993,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),22:25,07h 10m,05:35,2.4,INR 595,39 Seats available
12994,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,City Land Travels,A/C Seater/Sleeper (2+1),23:50,07h 40m,07:30,2.4,619,30 Seats available


In [674]:
Red_bus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12996 entries, 0 to 12995
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Route_name      12996 non-null  object
 1   Route_link      12996 non-null  object
 2   Bus_name        12996 non-null  object
 3   Bus_type        12996 non-null  object
 4   Depature_time   12996 non-null  object
 5   Duration        12996 non-null  object
 6   Reaching_time   12996 non-null  object
 7   Rating          12996 non-null  object
 8   Price           12996 non-null  object
 9   Seat_Available  12996 non-null  object
dtypes: object(10)
memory usage: 1015.4+ KB


In [677]:
Red_bus_df[Red_bus_df['Rating']=='New']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
90,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,KPS Buses,A/C Sleeper (2+1),21:15,07h 15m,04:30,New,2700,27 Seats available
100,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri Sai Padmavathi Travels,A/C Seater / Sleeper (2+1),23:45,05h 25m,05:10,New,INR 650,35 Seats available
101,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Naveen Tours and Travels,A/C Seater/Sleeper (2+1),14:50,07h 25m,22:15,New,INR 3999,44 Seats available
102,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Naveen Tours and Travels,A/C Seater/Sleeper (2+1),22:00,08h 15m,06:15,New,INR 3898,44 Seats available
103,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Naveen Tours and Travels,A/C Seater/Sleeper (2+1),22:00,06h 15m,04:15,New,INR 3999,44 Seats available
...,...,...,...,...,...,...,...,...,...,...
12649,Chandigarh to Kullu,https://www.redbus.in/bus-tickets/chandigarh-t...,Ankush travels,Volvo A/C Semi Sleeper (2+2),22:00,04h 30m,02:30,New,INR 999,41 Seats available
12687,Delhi to Bilaspur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bil...,VT Bus,Scania Multi-Axle AC Semi Sleeper (2+2),21:00,06h 00m,03:00,New,801,41 Seats available
12796,Bilaspur (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bilaspur-him...,VT Bus,Scania Multi-Axle AC Semi Sleeper (2+2),22:50,06h 40m,05:30,New,801,42 Seats available
12844,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Sonu Nath Bus Service,VE A/C Semi Sleeper (2+2),18:30,11h 15m,05:45,New,INR 2500,2 Seats available


In [689]:
Red_bus_df['Rating'] = pd.to_numeric(Red_bus_df['Rating'],errors='coerce')
rating_mean = round(Red_bus_df['Rating'].mean(),1)
Red_bus_df['Rating'].fillna(rating_mean,inplace=True)
Red_bus_df['Rating'] = Red_bus_df['Rating'].astype(float)

C:\Users\nagar\AppData\Local\Temp\ipykernel_9040\1755810501.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Red_bus_df['Rating'].fillna(rating_mean,inplace=True)


In [691]:
# Red_bus_df.dtypes

In [696]:
Red_bus_df['Price'] = Red_bus_df['Price'].str.replace('INR','')
Red_bus_df['Price'] = Red_bus_df['Price'].astype(float)

In [697]:
Red_bus_df.dtypes

Route_name         object
Route_link         object
Bus_name           object
Bus_type           object
Depature_time      object
Duration           object
Reaching_time      object
Rating            float64
Price             float64
Seat_Available     object
dtype: object

In [703]:
Red_bus_df['Seat_Available'] = Red_bus_df['Seat_Available'].str.replace(r' Seats? available', '', regex=True)
Red_bus_df['Seat_Available'] = Red_bus_df['Seat_Available'].astype(int)

In [704]:
Red_bus_df.dtypes

Route_name         object
Route_link         object
Bus_name           object
Bus_type           object
Depature_time      object
Duration           object
Reaching_time      object
Rating            float64
Price             float64
Seat_Available      int32
dtype: object

In [706]:
Red_bus_df

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:50,05h 45m,05:35,4.6,410.0,42
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),04:35,05h 55m,10:30,4.7,450.0,30
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),05:50,05h 55m,11:45,4.7,450.0,33
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdShyamoli Paribahan Pvt Ltd,Scania Multi-Axle AC Semi Sleeper (2+2),23:25,05h 45m,05:10,4.4,383.9,45
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),22:35,06h 40m,05:15,4.7,332.0,29
...,...,...,...,...,...,...,...,...,...,...
12991,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,CAPITAL HOLIDAYS,A/C Seater / Sleeper (2+2),21:30,06h 00m,03:30,2.1,539.0,39
12992,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),20:20,07h 10m,03:30,2.4,595.0,38
12993,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),22:25,07h 10m,05:35,2.4,595.0,39
12994,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,City Land Travels,A/C Seater/Sleeper (2+1),23:50,07h 40m,07:30,2.4,619.0,30


In [705]:
Red_bus_df.to_csv('Red_bus_df.csv')

# Connect to Database

In [708]:
myconnection = pymysql.connect(host='127.0.0.1',user='root',passwd='@neSre11')

In [709]:
myconnection.cursor()

In [ ]:
#csv reading
# df = pd.read_csv("Red_bus_df.csv")

In [716]:
# create_table_query = """
#     CREATE TABLE IF NOT EXISTS bus_routes (
#         id INT NOT NULL AUTO_INCREMENT PRIMARY KEY, 
#         route_name TEXT, 
#         route_link TEXT, 
#         bus_name TEXT, 
#         bus_type TEXT,
#         departing_time TIME, 
#         duration TEXT, 
#         reaching_time TIME,
#         star_rating FLOAT, 
#         price DECIMAL(10, 2), 
#         seats_available INT
#     );
#     """

#     # Execute the query
#     cursor.execute(create_table_query)
#     myconnection.commit()

In [723]:
query = """
    INSERT INTO redbus.bus_routes (route_name, route_link, bus_name, bus_type, departing_time, duration, reaching_time, star_rating, price, seats_available)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

In [724]:
for i in range(len(Red_bus_df)):
    data = tuple(Red_bus_df.iloc[i])      # Skipping the 'index: 0' column
#     print(data)
    myconnection.cursor().executemany(query, data)

In [726]:
myconnection.commit()

In [727]:
pd.read_sql_query('''select * from redbus.bus_routes''',myconnection)

C:\Users\nagar\AppData\Local\Temp\ipykernel_9040\3963494119.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''select * from redbus.bus_routes''',myconnection)


,id,route_name,route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seats_available
0,1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),0 days 23:50:00,05h 45m,0 days 05:35:00,4.6,410.0,42
1,2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),0 days 04:35:00,05h 55m,0 days 10:30:00,4.7,450.0,30
2,3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),0 days 05:50:00,05h 55m,0 days 11:45:00,4.7,450.0,33
3,4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdShyamoli Paribahan Pvt Ltd,Scania Multi-Axle AC Semi Sleeper (2+2),0 days 23:25:00,05h 45m,0 days 05:10:00,4.4,384.0,45
4,5,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),0 days 22:35:00,06h 40m,0 days 05:15:00,4.7,332.0,29
...,...,...,...,...,...,...,...,...,...,...,...
12991,12992,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,CAPITAL HOLIDAYS,A/C Seater / Sleeper (2+2),0 days 21:30:00,06h 00m,0 days 03:30:00,2.1,539.0,39
12992,12993,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),0 days 20:20:00,07h 10m,0 days 03:30:00,2.4,595.0,38
12993,12994,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),0 days 22:25:00,07h 10m,0 days 05:35:00,2.4,595.0,39
12994,12995,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...,City Land Travels,A/C Seater/Sleeper (2+1),0 days 23:50:00,07h 40m,0 days 07:30:00,2.4,619.0,30
